In [9]:
import struct
import serial
from math import pi
import sys
import glob
import serial


def serial_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result


class CamsenseX1:
    def __init__(self, port, baudrate, offset_angle = 16):
        self.ser = serial.Serial(
        port, baudrate,
        parity=serial.PARITY_NONE,
        stopbits=serial.STOPBITS_ONE,
        bytesize=serial.EIGHTBITS,
        timeout=None)
        self.ser.reset_input_buffer()
        self.package_header = b'\x55\xAA\x03\x08'
        self.angle_offset = offset_angle*pi/180
        self.data_buffer = {'angle_min': 0, 'angle_max': 0, 'speed': 0, 'ranges':[], 'intensities':[]}
        self.data_cnt = 0
        self.header_cnt = 0
        self.package_buffer = {'angle_min': 0, 'angle_max': 0, 'speed': 0, 'ranges':[], 'intensities':[]}
        self.package_cnt = 0

    # One angle value
    def read_angle(self):
        return (struct.unpack("<H", self.ser.read(2))[0]/64.0 - 640.0)*pi/180

    # One speed value (/60/64)
    def read_speed(self):
        return struct.unpack("<H", self.ser.read(2))[0]/3840*2*pi

    # One range value
    def read_range(self):
        return struct.unpack("<H", self.ser.read(2))[0]/32768*8

    # One quality value
    def read_intensity(self):
        return float(struct.unpack("<B", self.ser.read(1))[0])

    def read_crc(self):
        return struct.unpack("<H", self.ser.read(2))[0]

    # One data package
    def read_package(self):
        return struct.unpack("<BBBBHHHBHBHBHBHBHBHBHBHH", self.ser.read(36))

    # Only package data without header
    def read_package_data(self):
        return struct.unpack("<HHHBHBHBHBHBHBHBHBHH", self.ser.read(32))

    # Read until header is found, then parse package data 
    def get_package(self):
        self.ser.read_until(self.package_header)
        data = self.read_package_data()
        return {'angle_min': (data[1] / 64.0 - 640.0)*pi/180, 
                    'angle_max': (data[18] / 64.0 - 640.0)*pi/180, 
                    'speed': data[0] / 3840, 
                    'ranges': [data[i] / 32768 * 8 for i in [2, 4, 6, 8, 10, 12, 14, 16]], 
                    'intensities':[float(data[i]) for i in [3, 5, 7, 9, 11, 13, 15, 17]]}

    # Look for header and then read each data value
    def read_next(self):
        if self.data_cnt == 0:
            if(self.find_header()):
                self.data_cnt +=1
        elif self.data_cnt == 1:
            self.data_buffer['speed'] = self.read_speed()
            self.data_cnt +=1
        elif self.data_cnt == 2:	
            self.data_buffer['angle_min'] = self.read_angle() + self.angle_offset
            self.data_cnt +=1
        elif self.data_cnt in [3,5,7,9,11,13,15,17]:	
            self.data_buffer['ranges'].append(self.read_range())
            self.data_cnt +=1
        elif self.data_cnt in [4,6,8,10,12,14,16,18]:	
            self.data_buffer['intensities'].append(self.read_intensity())
            self.data_cnt +=1
        elif self.data_cnt == 19:	
            self.data_buffer['angle_max'] = self.read_angle() + self.angle_offset
            self.data_cnt +=1
        elif self.data_cnt == 20:	
            self.data_cnt = 0
            self.read_crc()
            return True

        return False

    # Look for header
    def find_header(self):
        bit = self.ser.read(1)
        if ((self.header_cnt == 0 and bit == self.package_header[0].to_bytes(1,byteorder ='little')) or 
            (self.header_cnt == 1 and bit == self.package_header[1].to_bytes(1,byteorder ='little')) or
            (self.header_cnt == 2 and bit == self.package_header[2].to_bytes(1,byteorder ='little')) or
            (self.header_cnt == 3 and bit == self.package_header[3].to_bytes(1,byteorder ='little'))):
            self.header_cnt += 1
        else:
            self.header_cnt = 0

        if self.header_cnt == 4:
            self.header_cnt = 0
            return True
        return False

    def get_data_buffer(self):
        return self.data_buffer

    def reset_data_buffer(self):
        self.data_buffer = {'angle_min': 0, 'angle_max': 0, 'speed': 0, 'ranges':[], 'intensities':[]}

    def pop_data_buffer(self):
        data = self.get_data_buffer()
        self.reset_data_buffer()
        return data	

    def get_next_scan(self):
        if self.read_next():
            return self.pop_data_buffer()	

    # Read n number of packages
    def read_n_packages(self, n):
        package = self.get_next_scan()
        if package:
            self.package_cnt += 1

            self.package_buffer['speed'] = package['speed']
            self.package_buffer['ranges'] += package['ranges']
            self.package_buffer['intensities'] += package['intensities']
            if self.package_cnt == 1:
                self.package_buffer['angle_min'] = package['angle_min']
            elif self.package_cnt >= n:
                self.package_buffer['angle_max'] = package['angle_max']
                self.package_buffer['speed'] /= self.package_cnt
                return True		
        return False

        ''' Simpler, less robust version
        package = self.get_package(self)
        self.package_cnt += 1

        self.package_buffer['speed'] = package['speed']
        self.package_buffer['ranges'] += package['ranges']
        self.package_buffer['intensities'] += package['intensities']
        if self.package_cnt == 1:
            self.package_buffer['angle_min'] = package['angle_min']
        elif self.package_cnt >= n:
            self.package_buffer['angle_max'] = package['angle_max']
            #self.package_buffer['speed'] /= self.package_cnt
            return True			
        return False
        '''

    def get_package_buffer(self):
        return self.package_buffer

    def reset_package_buffer(self):
        self.package_cnt = 0
        self.package_buffer = {'angle_min': 0, 'angle_max': 0, 'speed': 0, 'ranges':[], 'intensities':[]}

    def pop_package_buffer(self):
        data = self.get_package_buffer()
        self.reset_package_buffer()
        return data

    def get_full_scan(self):
        if self.read_n_packages(50):
            return self.pop_package_buffer()

In [11]:
serial_ports()

['COM3']

In [ ]:
cam = CamsenseX1('COM3', 115200)
while True:
    print(cam.get_next_scan())
    print(cam.get_full_scan())